We check the decomposition of a multi-controlled Z gate obtained via ZX calculus (Table III in M. Schumann et al "Bridging wire and gate cutting with ZX-calculus" (2025))

In [2]:
import numpy as np
from util import return_ptm

In [3]:
def apply_mcphase_theta(op: np.ndarray, params: dict):
    n = int(np.log2(op.shape[1]))
    if op.shape[1] != 2**n:
        raise ValueError("Matrix shape error: the matrix dimension must be a power of 2")
    theta = params["theta"]
    mcphase = np.identity(2**n, dtype=complex)
    mcphase[-1, -1] = np.exp(1j*theta)
    return mcphase@op@mcphase.conj().T 

Now let us focus on the case $n=4$ and $m = m' = 2$

In [4]:
num_qubits = 4
m = 2
m_prime = num_qubits - m

In [5]:
target_ptm_mcz = return_ptm(apply_mcphase_theta, num_qubits, params={"theta": np.pi})

Now we obtain the PTMs for each term in the decomposition
1. $\mathcal{MCP}^{(m)}(\pi/2) \otimes \mathcal{MCP}^{(m')}(\pi/2)$

In [6]:
deco_dict = {}
deco_dict["1"] = {}
deco_dict["1"]["q"] = 1/2
deco_dict["1"]["ptm"] = np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": np.pi/2}), return_ptm(apply_mcphase_theta, m_prime, params={"theta": np.pi/2}))

2. $\mathcal{MCP}^{(m)}(-\pi/2) \otimes \mathcal{MCP}^{m'}(-\pi/2)$

In [7]:
deco_dict["2"] = {}
deco_dict["2"]["q"] = 1/2
deco_dict["2"]["ptm"] = np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": -np.pi/2}), return_ptm(apply_mcphase_theta, m_prime, params={"theta": -np.pi/2}))

3. $\mathcal{E}_{\mathrm{MCZ}-\ket{\pm}_a}^{(m)} \otimes \mathcal{I}^{\otimes m'} $

In [ ]:
def apply_mcz_ancilla_measurement(op: np.ndarray, params=None):
    n = int(np.log2(op.shape[1]))
    if op.shape[1] != 2**n:
        raise ValueError("Matrix shape error: the matrix dimension must be a power of 2")
    ancilla_state = 1/np.sqrt(2)*np.array([[1], [1]])
    rho_plus = 1/2*np.array([[1, 1], [1, 1]])
    rho_minus = 1/2*np.array([[1, -1], [-1, 1]])
    rho_ancilla_state = ancilla_state@ancilla_state.conj().T
    op_with_ancilla = np.kron(op, rho_ancilla_state)
    op_with_ancilla_mcz = apply_mcphase_theta(op_with_ancilla, {"theta": np.pi})
    proj_plus = np.kron(np.identity(2**n), rho_plus)
    proj_minus = np.kron(np.identity(2**n), rho_minus)
    op_proj_plus = proj_plus@op_with_ancilla_mcz@proj_plus
    op_proj_minus = proj_minus@op_with_ancilla_mcz@proj_minus 
    op_plus_trace = np.trace(op_proj_plus.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3)
    op_minus_trace = np.trace(op_proj_minus.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3)
    return op_plus_trace - op_minus_trace
    
    

In [9]:
deco_dict["3"] = {}
deco_dict["3"]["q"] = 1/2
deco_dict["3"]["ptm"] = np.kron(return_ptm(apply_mcz_ancilla_measurement, m, params=None), return_ptm(apply_mcphase_theta, m_prime, params={"theta": 2*np.pi}))

4. $\mathcal{E}_{\mathrm{MCZ}-\ket{\pm}_a}^{(m)} \otimes \mathcal{MCZ}^{( m')} $

In [10]:
deco_dict["4"] = {}
deco_dict["4"]["q"] = -1/2
deco_dict["4"]["ptm"] = np.kron(return_ptm(apply_mcz_ancilla_measurement, m, params=None), return_ptm(apply_mcphase_theta, m_prime, params={"theta": np.pi}))

5. $\mathcal{I}^{\otimes m} \otimes \mathcal{E}_{\mathrm{MCZ}-\ket{\pm}_a}^{(m')}$

In [11]:
deco_dict["5"] = {}
deco_dict["5"]["q"] = 1/2
deco_dict["5"]["ptm"] = np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": 2*np.pi}), return_ptm(apply_mcz_ancilla_measurement, m_prime, params=None))

6. $\mathcal{MCZ}^{(m)}\otimes \mathcal{E}_{\mathrm{MCZ}-\ket{\pm}_a}^{(m')} $

In [12]:
deco_dict["6"] = {}
deco_dict["6"]["q"] = -1/2
deco_dict["6"]["ptm"] =  np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": np.pi}), return_ptm(apply_mcz_ancilla_measurement, m_prime, params=None))

In [13]:
cut = sum([deco_dict[key]["q"]*deco_dict[key]["ptm"] for key in deco_dict.keys()])

In [14]:
np.max(np.abs(target_ptm_mcz - cut))

np.float64(1.1102230246251565e-16)

which shows that the decomposition is correct!